## Imports


In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
from dotenv import load_dotenv
from os import getenv
from pathlib import Path

load_dotenv()

import findspark

findspark.init()

In [2]:
CWD: Path = Path("/app/")
EXAMPLE_INPUT_PATH: Path = CWD / Path("./coding_challenge_files/example_input.txt")

## TAST PART TWO MYSQL DB


### Database Config


First we setup the connector driver path


In [3]:
MYSQL_CONNECTOR_FILENAME: str = "mysql-connector-j-8.2.0.jar"
MYSQL_CONNECTOR_PATH: str = f"/app/mysql_connector/{MYSQL_CONNECTOR_FILENAME}"
TABLE_NAME: str = "INSTRUMENT_PRICE_MODIFIER"

We check if hte connector path is correct


In [4]:
Path(MYSQL_CONNECTOR_PATH).exists()

True

Then we move the connector driver in the proper place in order to be recognizable by pyspark


In [5]:
findspark.add_jars(MYSQL_CONNECTOR_PATH)

We setup the relevant credentials for the database connection


In [6]:
# database connection info
DB_CON_DICT = dict(
    user=getenv("MYSQL_ROOT_USER"),
    password=getenv("MYSQL_ROOT_PASSWORD"),
    host=getenv("HOST"),
    port=int(getenv("MYSQL_DOCKER_PORT")),  # type: ignore
    database=getenv("MYSQL_DATABASE"),
)

DB_CON_DICT

{'user': 'root',
 'password': 'example',
 'host': 'db',
 'port': 3306,
 'database': 'mydb'}

We also setup the pyspark specific format we need for the database connection


In [7]:
# Configure MySQL connection properties
MYSQL_PROPERTIES = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "url": "jdbc:mysql://{host}:{port}/{database}".format(**DB_CON_DICT),  # type: ignore
    "user": DB_CON_DICT["user"],  # type: ignore
    "password": DB_CON_DICT["password"],  # type: ignore
}

MYSQL_PROPERTIES

{'driver': 'com.mysql.cj.jdbc.Driver',
 'url': 'jdbc:mysql://db:3306/mydb',
 'user': 'root',
 'password': 'example'}

We test the database connection


In [8]:
from tests.src import test_mysql_conx

In [9]:
# test database connection
test_mysql_conx(**DB_CON_DICT)  # type: ignore

Connection Success


We create a test table with some values for pyspark database test


In [10]:
from tests.src import table_preparation

In [11]:
table_preparation()

Table 'test_table' deleted successfully.
Table 'test_table' created successfully.
Sample data inserted successfully.


We test the pyspark session against this test_table


In [12]:
from tests.src import test_pyspark_db_conx

In [13]:
test_pyspark_db_conx()

MYSQL_PROPERTIES={'driver': 'com.mysql.cj.jdbc.Driver', 'url': 'jdbc:mysql://db:3306/mydb', 'user': 'root', 'password': 'example'}
MYSQL driver path existence: True


24/01/11 17:37:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+-----+
| id| name|
+---+-----+
|  1| John|
|  2|Alice|
|  3|  Bob|
+---+-----+



We clean up the test_table


In [14]:
from tests.src import drop_table

In [15]:
drop_table()

Table 'test_table' deleted successfully.


### `So as part of your task we would like you to set up a database with only one table, called INSTRUMENT_PRICE_MODIFIER with the following columns:`

-   ID (primary key)
-   NAME (instrument name as read from the input file)
-   MULTIPLIER - double value


In [16]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("DatabaseConnection")
    .config("spark.jars", MYSQL_CONNECTOR_PATH)
    .config("spark.driver.extraClassPath", MYSQL_CONNECTOR_PATH)
    .config("spark.executor.extraClassPath", MYSQL_CONNECTOR_PATH)
    .getOrCreate()
)

Next we create the schema for the table


In [17]:
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
)

# Define the schema
SCHEMA_DB = StructType(
    [
        StructField(
            "ID", IntegerType(), False
        ),  # False indicates that the field is not nullable
        StructField(
            "NAME", StringType(), True
        ),  # True indicates that the field is nullable
        StructField("MULTIPLIER", DoubleType(), True),
    ]
)

For testing purposes we generate some dummy data, WITHOUT INSTRUMENT1, INSTRUMENT3 for the sake of realistic example


In [18]:
# from random import uniform, randint

# Generate test data
# num_rows = 20
# data = [
#     (i, f"INSTRUMENT{choice([2,4,5,6])}", round(uniform(1.0, 10.0), 2))
#     for i in range(1, num_rows + 1)
# ]

data: list[tuple[int, str, float]] = [
    (1, "INSTRUMENT5", 5.19),
    (2, "INSTRUMENT4", 5.05),
    (3, "INSTRUMENT2", 4.4),
    (4, "INSTRUMENT4", 2.25),
    (5, "INSTRUMENT5", 1.75),
    (6, "INSTRUMENT6", 9.91),
    (7, "INSTRUMENT4", 4.5),
    (8, "INSTRUMENT2", 9.24),
    (9, "INSTRUMENT6", 5.83),
    (10, "INSTRUMENT5", 1.34),
    (11, "INSTRUMENT6", 8.89),
    (12, "INSTRUMENT2", 2.59),
    (13, "INSTRUMENT5", 4.04),
    (14, "INSTRUMENT4", 8.58),
    (15, "INSTRUMENT2", 8.64),
    (16, "INSTRUMENT2", 3.99),
    (17, "INSTRUMENT2", 6.82),
    (18, "INSTRUMENT2", 7.7),
    (19, "INSTRUMENT4", 4.44),
    (20, "INSTRUMENT4", 8.01),
]

In [19]:
df = spark.createDataFrame(data, schema=SCHEMA_DB).orderBy("ID")

df.printSchema()

df.show(5)

root
 |-- ID: integer (nullable = false)
 |-- NAME: string (nullable = true)
 |-- MULTIPLIER: double (nullable = true)



+---+-----------+----------+
| ID|       NAME|MULTIPLIER|
+---+-----------+----------+
|  1|INSTRUMENT5|      5.19|
|  2|INSTRUMENT4|      5.05|
|  3|INSTRUMENT2|       4.4|
|  4|INSTRUMENT4|      2.25|
|  5|INSTRUMENT5|      1.75|
+---+-----------+----------+
only showing top 5 rows



In [20]:
# Write the DataFrame to MySQL
df.write.jdbc(
    url=MYSQL_PROPERTIES["url"],
    table=TABLE_NAME,
    mode="overwrite",  # or "append" if needed
    properties=MYSQL_PROPERTIES,
)

### Determination of the final value


#### 1 `read the line from the input file;`


In [21]:
# Specify the path to the .txt file
txt_file_path: str = f"{EXAMPLE_INPUT_PATH}"

# Define the schema with StringType for DATE initially
schema = StructType(
    [
        StructField(name="INSTRUMENT_NAME", dataType=StringType(), nullable=True),
        StructField(name="DATE", dataType=StringType(), nullable=True),
        StructField(name="VALUE", dataType=DoubleType(), nullable=True),
    ]
)

# Read the .txt file into a PySpark DataFrame
extr = spark.read.option("delimiter", ",").csv(
    txt_file_path, header=False, schema=schema
)

# transform to dataframe
df_txt = extr.toDF("INSTRUMENT_NAME", "DATE", "VALUE")

In [22]:
df_txt.printSchema()

df_txt.show(5)

root
 |-- INSTRUMENT_NAME: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- VALUE: double (nullable = true)



+---------------+-----------+------+
|INSTRUMENT_NAME|       DATE| VALUE|
+---------------+-----------+------+
|    INSTRUMENT1|01-Jan-1996|2.4655|
|    INSTRUMENT1|02-Jan-1996|2.4685|
|    INSTRUMENT1|03-Jan-1996| 2.473|
|    INSTRUMENT1|04-Jan-1996|2.4845|
|    INSTRUMENT1|05-Jan-1996|2.4868|
+---------------+-----------+------+
only showing top 5 rows



Assuming that the data is sanitized and preprocessed as in task_part_1.ipynb we proceed with the objectives of the second part and extract all rows

In [23]:
rows = df_txt.collect()
rows[:5]

[Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='01-Jan-1996', VALUE=2.4655),
 Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='02-Jan-1996', VALUE=2.4685),
 Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='03-Jan-1996', VALUE=2.473),
 Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='04-Jan-1996', VALUE=2.4845),
 Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='05-Jan-1996', VALUE=2.4868)]

#### Constructing the Solution

`query the database to see if there is an entry for the <INSTRUMENT_NAME> you read in the 0st step;`

`if there is - multiply the original <VALUE> by the factor you found in the step 2;`

`if there is no entry - simply use the original <VALUE> from the file`


The rough idea is the following for a couple of rows. For our example we cherry pick a couple of rows from each INSTRUMENT name we have on the .txt file


In [24]:
l_1 = [row for row in rows if row["INSTRUMENT_NAME"] == "INSTRUMENT1"][:2]
l_1

[Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='01-Jan-1996', VALUE=2.4655),
 Row(INSTRUMENT_NAME='INSTRUMENT1', DATE='02-Jan-1996', VALUE=2.4685)]

In [25]:
l_2 = [row for row in rows if row["INSTRUMENT_NAME"] == "INSTRUMENT2"][:2]
l_2

[Row(INSTRUMENT_NAME='INSTRUMENT2', DATE='22-Feb-1996', VALUE=9.326787847),
 Row(INSTRUMENT_NAME='INSTRUMENT2', DATE='23-Feb-1996', VALUE=9.321527686)]

In [26]:
l_3 = [row for row in rows if row["INSTRUMENT_NAME"] == "INSTRUMENT3"][:2]
l_3

[Row(INSTRUMENT_NAME='INSTRUMENT3', DATE='31-May-2012', VALUE=78.5325),
 Row(INSTRUMENT_NAME='INSTRUMENT3', DATE='01-Jun-2012', VALUE=78.2655)]

In [27]:
example_list = l_1 + l_2 + l_3

We engineered our database to not include INSTRUMENT1 for this example to be realistic


In [28]:
# create an empty dataframe
result_df = spark.createDataFrame([], schema=SCHEMA_DB)

# let's see the first 10
for row in example_list:
    # check if the name is empty
    if name := row["INSTRUMENT_NAME"]:
        # if not we proceed with querying the database
        query: str = f"SELECT * FROM {TABLE_NAME} WHERE NAME = '{name}'"

        # Query the database
        result_df = (
            spark.read.format("jdbc").options(**MYSQL_PROPERTIES, query=query).load()
        )
        print(f"{name=}")
        print(f'Initial value: {row["VALUE"]}')

    # is there is match in the database
    if not result_df.isEmpty():
        print(f"Multiplier found: {result_df.collect()[0]['MULTIPLIER']}")
        final_value = row["VALUE"] * result_df.collect()[0]["MULTIPLIER"]
        print(f"{final_value=}")
    # if there is no match in the database
    else:
        print("No Multiplier found")
        final_value = row["VALUE"]
        print(f"{final_value=}")
    print("-----------------------------")

name='INSTRUMENT1'
Initial value: 2.4655
No Multiplier found
final_value=2.4655
-----------------------------
name='INSTRUMENT1'
Initial value: 2.4685
No Multiplier found
final_value=2.4685
-----------------------------
name='INSTRUMENT2'
Initial value: 9.326787847


Multiplier found: 4.4
final_value=41.0378665268
-----------------------------
name='INSTRUMENT2'
Initial value: 9.321527686
Multiplier found: 4.4
final_value=41.0147218184
-----------------------------
name='INSTRUMENT3'
Initial value: 78.5325
No Multiplier found
final_value=78.5325
-----------------------------
name='INSTRUMENT3'
Initial value: 78.2655
No Multiplier found
final_value=78.2655
-----------------------------


So the rough idea works just fine. Let us refine it by properly introducing a proper query mechanism according to the specifications.


Since the query is pretty simple and the table is supposed to be GBs it is much more efficient to directly query the database with the instrument name in order to obtain our result.

-   This really depends on how powerful is the pyspark cluster or the database itself.
-   We assume that direct query to the database consumes the least resources in this case.

Regarding the time frequency of our queries, we will use a guard condition that will return the results of the previous query if we are within the 5 seconds time-window and the previous instrument name is the same as the current instrument name. Otherwise a new query will issued on the database.

This construction will be realized using the notion of a closure. A closure very roughly is a function with memory.


In [29]:
from datetime import datetime, timezone

DB_MIN_UPDATE_TIME = 5  # secs


def handle_query(instrument_name):
    """helper function to query a specific instrument in the db"""
    query: str = f"SELECT * FROM {TABLE_NAME} WHERE NAME = '{instrument_name}'"

    # Query the database
    return spark.read.format("jdbc").options(**MYSQL_PROPERTIES, query=query).load()


def query_db_closure(verbose: bool = False):
    result_df = spark.createDataFrame([], schema=SCHEMA_DB)
    last_time: datetime = datetime.now(timezone.utc)
    last_instrument_name: str = ""
    
    # this is a bookmark
    # of how many times the function
    # query_db has been called
    call_counter = 1

    def query_db(instrument_name: str):
        # this is to make the variables
        # available to the upper scope of
        # which is query_db_closure
        nonlocal result_df, last_time, last_instrument_name, call_counter

        current_time: datetime = datetime.now(timezone.utc)
        timediff: int = (current_time - last_time).seconds

        # printing logs
        if verbose:
            print("----query_db----")
            print(f"{last_instrument_name=}, current_instrument_name={instrument_name}")
            print(f"{call_counter=}")

        # this is a guard condition that ensures
        # we query the database a maximum of once
        # every 5 seconds for the same instrument
        if timediff <= DB_MIN_UPDATE_TIME and (last_instrument_name == instrument_name):
            if verbose:
                current_time_str: str = datetime.strftime(
                    current_time, "%d/%m/%Y, %H:%M:%S"
                )
                print(f"SAME Q {current_time_str}, {timediff=}")
                print(" ")
            last_time = current_time
            last_instrument_name = instrument_name
            call_counter += 1
            return result_df

        # printing logs
        if verbose:
            current_time_str: str = datetime.strftime(
                current_time, "%d/%m/%Y, %H:%M:%S"
            )
            print(f"NEW Q {current_time_str}, {timediff=}")
            print(" ")

        # query database
        result_df = handle_query(instrument_name)

        last_time = current_time
        last_instrument_name = instrument_name
        call_counter += 1
        return result_df

    return query_db

Let's test this first with the exact same instruments all over


In [30]:
# dummy_instrument_list = [f"INSTRUMENT{choice([1, 1])}" for i in range(1, 100)]

dummy_instrument_list: list[str] = [
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
    "INSTRUMENT1",
]

dummy_instrument_list

['INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1',
 'INSTRUMENT1']

We should encouter NEW Q (new querries) only when timediff > 5


In [31]:
import time
from random import randint

query_db = query_db_closure(verbose=True)

for instrument in dummy_instrument_list:
    # artificial time delay
    # in order to witness
    # subsequent time intervals > 5secs
    time.sleep(randint(1, 10))
    _ = query_db(instrument)

----query_db----
last_instrument_name='', current_instrument_name=INSTRUMENT1
call_counter=1
NEW Q 11/01/2024, 17:37:32, timediff=6
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=2
NEW Q 11/01/2024, 17:37:41, timediff=9
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=3
NEW Q 11/01/2024, 17:37:50, timediff=9
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=4
SAME Q 11/01/2024, 17:37:54, timediff=4
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=5
SAME Q 11/01/2024, 17:37:59, timediff=5
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=6
SAME Q 11/01/2024, 17:38:02, timediff=3
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=7
SAME Q 11/01/2024, 17:38:04, timed

everything seems to be working as it should, we get a new query only if timediff>5secs which means 5 seconds pass after the previous query


In the same manner for random instruments


In [32]:
# dummy_instrument_list = [f"INSTRUMENT{choice([1,2,3])}" for i in range(1, 10)]

dummy_instrument_list: list[str] = [
    "INSTRUMENT2",
    "INSTRUMENT2",
    "INSTRUMENT2",
    "INSTRUMENT1",
    "INSTRUMENT3",
    "INSTRUMENT1",
    "INSTRUMENT2",
    "INSTRUMENT3",
    "INSTRUMENT3",
]

dummy_instrument_list

['INSTRUMENT2',
 'INSTRUMENT2',
 'INSTRUMENT2',
 'INSTRUMENT1',
 'INSTRUMENT3',
 'INSTRUMENT1',
 'INSTRUMENT2',
 'INSTRUMENT3',
 'INSTRUMENT3']

In [33]:
query_db = query_db_closure(verbose=True)

for instrument in dummy_instrument_list:
    # artificial time delay
    # in order to witness
    # subsequent time intervals > 5secs
    time.sleep(randint(1, 10))
    _ = query_db(instrument)

----query_db----
last_instrument_name='', current_instrument_name=INSTRUMENT2
call_counter=1
NEW Q 11/01/2024, 17:38:55, timediff=6
 
----query_db----
last_instrument_name='INSTRUMENT2', current_instrument_name=INSTRUMENT2
call_counter=2
NEW Q 11/01/2024, 17:39:03, timediff=8
 
----query_db----
last_instrument_name='INSTRUMENT2', current_instrument_name=INSTRUMENT2
call_counter=3
NEW Q 11/01/2024, 17:39:11, timediff=8
 
----query_db----
last_instrument_name='INSTRUMENT2', current_instrument_name=INSTRUMENT1
call_counter=4
NEW Q 11/01/2024, 17:39:16, timediff=5
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT3
call_counter=5
NEW Q 11/01/2024, 17:39:22, timediff=6
 
----query_db----
last_instrument_name='INSTRUMENT3', current_instrument_name=INSTRUMENT1
call_counter=6
NEW Q 11/01/2024, 17:39:32, timediff=10
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT2
call_counter=7
NEW Q 11/01/2024, 17:39:34, timediff

As we can see for different subsequent instrument names our closure works as intended without regard for the time constraint of 5 secs but it does respect it in case of same subsequent instruments.


### Putting everything together


Running this code we expect multipliers only for INSTRUMENT2 since our data in the database has been engineered to not contain INSTRUMENT1, INSTRUMENT3.

In [34]:
query_db = query_db_closure(True)
final_values = []

for row in example_list:
    # artificial time delay
    # in order to witness
    # subsequent time intervals > 5secs
    time.sleep(randint(1, 10))

    if instrument := row["INSTRUMENT_NAME"]:
        result_df = query_db(instrument)
        old_value = row["VALUE"]
        print(f"{instrument=}")
        print(f'Initial value: {old_value}')
        
        # is there is match in the database
        if not result_df.isEmpty():
            print(f"Multiplier found: {result_df.collect()[0]['MULTIPLIER']}")
            final_value = old_value * result_df.collect()[0]["MULTIPLIER"]
            print(f"{final_value=}")
        # if there is no match in the database
        else:
            print("No Multiplier found")
            final_value = old_value
            print(f"{final_value=}")
        print("\n")
        final_values.append((instrument, old_value, final_value))
final_values

----query_db----
last_instrument_name='', current_instrument_name=INSTRUMENT1
call_counter=1
NEW Q 11/01/2024, 17:39:49, timediff=1
 
instrument='INSTRUMENT1'
Initial value: 2.4655
No Multiplier found
final_value=2.4655


----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=2
SAME Q 11/01/2024, 17:39:53, timediff=4
 
instrument='INSTRUMENT1'
Initial value: 2.4685
No Multiplier found
final_value=2.4685


----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT2
call_counter=3
NEW Q 11/01/2024, 17:39:59, timediff=6
 
instrument='INSTRUMENT2'
Initial value: 9.326787847
Multiplier found: 4.4
final_value=41.0378665268


----query_db----
last_instrument_name='INSTRUMENT2', current_instrument_name=INSTRUMENT2
call_counter=4
SAME Q 11/01/2024, 17:40:04, timediff=5
 
instrument='INSTRUMENT2'
Initial value: 9.321527686
Multiplier found: 4.4
final_value=41.0147218184


----query_db----
last_instrument_name='INSTRUMENT

[('INSTRUMENT1', 2.4655, 2.4655),
 ('INSTRUMENT1', 2.4685, 2.4685),
 ('INSTRUMENT2', 9.326787847, 41.0378665268),
 ('INSTRUMENT2', 9.321527686, 41.0147218184),
 ('INSTRUMENT3', 78.5325, 78.5325),
 ('INSTRUMENT3', 78.2655, 78.2655)]

Finally we refactor and modularize the calculation

In [38]:
def final_value_calc_row(row, query_db):
    '''row is read from the .txt file and query_db is the function coming from the closure query_db_closure'''
    instrument = row['INSTRUMENT_NAME']
    result_df = query_db(instrument)
    old_value = row["VALUE"]
    
    if result_df.isEmpty():
        return old_value
    return old_value * result_df.collect()[0]["MULTIPLIER"]

In [41]:
query_db = query_db_closure(True)
final_values = []

for row in example_list:
    time.sleep(randint(1, 10))

    if instrument := row["INSTRUMENT_NAME"]:
        final_values.append(final_value_calc_row(row, query_db))
        
final_values

----query_db----
last_instrument_name='', current_instrument_name=INSTRUMENT1
call_counter=1
NEW Q 11/01/2024, 18:42:41, timediff=2
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT1
call_counter=2
SAME Q 11/01/2024, 18:42:44, timediff=3
 
----query_db----
last_instrument_name='INSTRUMENT1', current_instrument_name=INSTRUMENT2
call_counter=3
NEW Q 11/01/2024, 18:42:54, timediff=10
 
----query_db----
last_instrument_name='INSTRUMENT2', current_instrument_name=INSTRUMENT2
call_counter=4
SAME Q 11/01/2024, 18:42:56, timediff=2
 
----query_db----
last_instrument_name='INSTRUMENT2', current_instrument_name=INSTRUMENT3
call_counter=5
NEW Q 11/01/2024, 18:43:02, timediff=6
 
----query_db----
last_instrument_name='INSTRUMENT3', current_instrument_name=INSTRUMENT3
call_counter=6
NEW Q 11/01/2024, 18:43:11, timediff=9
 


[2.4655, 2.4685, 41.0378665268, 41.0147218184, 78.5325, 78.2655]